data that can be downloaded at:
https://www.kaggle.com/c/dogs-vs-cats/data

Folder structure:
- created a data/ folder
- created train/ and validation/ subfolders inside data/
- created cats/ and dogs/ subfolders inside train/ and validation/
- put the cat pictures index 0-999 in data/train/cats
- put the cat pictures index 1000-1400 in data/validation/cats
- put the dogs pictures index 0-999 in data/train/dogs
- put the dog pictures index 1000-1400 in data/validation/dogs
So that we have 1000 training examples for each class, and 400 validation examples for each class.
In summary, this is our directory structure:
```
data/
    train/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...
    validation/
        dogs/
            dog001.jpg
            dog002.jpg
            ...
        cats/
            cat001.jpg
            cat002.jpg
            ...

In [ ]:
#import required libraries
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')


Mounted at /content/gdrive/


In [ ]:
# dimensions of our images.
img_width, img_height = 150, 150

train_data_dir = '/content/gdrive/MyDrive/data/train'
validation_data_dir = '/content/gdrive/MyDrive/data/validation'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

#model, simple stack of 3 convolution layers with a ReLU activation and followed by max-pooling layers

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

## Data pre-processing and data augmentation

In [ ]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 802 images belonging to 2 classes.


## Training a small convnet

In [ ]:
model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5') #save the model weights

<ipython-input-9-240a007c2074>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(


Epoch 1/50
125/125 [==============================] - 365s 3s/step - loss: 0.7074 - accuracy: 0.4935 - val_loss: 0.6898 - val_accuracy: 0.5000
Epoch 2/50
125/125 [==============================] - 89s 709ms/step - loss: 0.6930 - accuracy: 0.5240 - val_loss: 0.6880 - val_accuracy: 0.5013
Epoch 3/50
125/125 [==============================] - 87s 699ms/step - loss: 0.6849 - accuracy: 0.5570 - val_loss: 0.6447 - val_accuracy: 0.6438
Epoch 4/50
125/125 [==============================] - 88s 707ms/step - loss: 0.6524 - accuracy: 0.6225 - val_loss: 0.6491 - val_accuracy: 0.5738
Epoch 5/50
125/125 [==============================] - 87s 697ms/step - loss: 0.6282 - accuracy: 0.6450 - val_loss: 0.5779 - val_accuracy: 0.6925
Epoch 6/50
125/125 [==============================] - 87s 695ms/step - loss: 0.6129 - accuracy: 0.6710 - val_loss: 0.5873 - val_accuracy: 0.6825
Epoch 7/50
125/125 [==============================] - 88s 700ms/step - loss: 0.6017 - accuracy: 0.6845 - val_loss: 0.5912 - val_accu

## Conclusion
1. Got validation accuracy of 0.77 after 50 epochs.
2. the variance of the validation accuracy is fairly high, because accuracy is a high-variance metric and because we only use 800 validation samples.
3. A good validation strategy in such cases would be to do k-fold cross-validation, but this would require training k models for every evaluation round.